# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [ ]:
# import required libraries
import sys
# print(sys.path)
# Adding a path so that it finds the package for numpy
sys.path.append('/usr/local/lib/python3.7/site-packages')
import numpy as np
import pandas as pd

Next, import `Orders` from [here](https://drive.google.com/file/d/17oJ9NNbwJ8Wv4Jp-mLFiC59k72Brdyha/view?usp=sharing) and place it in the provided data folder. You will access the data folder with a relative path.
Load `Orders` into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [ ]:
# your code here

orders = pd.read_csv('../your-code/data/Orders.csv')
orders.head()


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# your code here

#Sub problem 1
# Use groupby sum to aggregate by unique customer

ord_cust = orders.groupby(['CustomerID']).sum()
ord_cust.head()

# I could use the following to have the full detail by customer of all orders
# orders.groupby(['CustomerID','amount_spent']).sum()


In [ ]:
# Subproblem 2

# Use quantile method to identify the treshold of VIP (95th) and Preferred segments
# ord_cust.quantile([.75, .95], axis = 0)


# Create a sorted framework on amount spent
ord_cust_sorted = ord_cust.sort_values(['amount_spent'], ascending = False)
ord_cust_sorted.quantile([.75, .95, 1], axis = 0)

cust_labels = ['All_Others', 'Preferred', 'VIP']
cutoffs = [0,1661.640,5840.182,280206.020]
bins['Customer_Segment'] = pd.cut(ord_cust_sorted['amount_spent'], cutoffs, labels=cust_labels)
bins.groupby(['Customer_Segment']).sum

# df.groupby(bins)['Value'].agg(['count', 'sum'])

# cutoffs = [7,14,21,23,30,40]
# bins = pd.cut(data['Combined MPG'],cutoffs, labels=mpg_labels)
# bins.head(10)


# pokemon.loc[pokemon['A/D Ratio'] == 0.043478260869565216]


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# your code here

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here